In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
#import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/Desktop/PHD2022/Autumn/CSE 579/CS_project/LQ_Game_Solver/CS_project/Project.toml`


In [12]:
using LinearAlgebra
using Plots
using SparseArrays
#using ControlSystems

In [3]:
c = 0.0
m = 1.0
function point_mass(state, u)
    x = state[1]                #x_1 = ẋ
    ẋ = state[2]
    ẍ = -(c/m)*x[2] + u[1]/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m 
    y = state[3]
    ẏ = state[4]
    ÿ = -(c/m)*x[4] + u[2]/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m 
    # dx[3] = x[3]
    # dx[4] = -(c/m)*x[3] + u(t)/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m
    return [ẋ; ẍ; ẏ; ÿ]
end

point_mass (generic function with 1 method)

#### Point Mass Discrete

$$ \frac{d}{dt}x = Ax + \sum Bu$$

Single 2D point mass:
$$\frac{d}{dt}\begin{bmatrix} x \\ y \\ \dot{x} \\ \dot{y}\end{bmatrix} =  
 \begin{bmatrix} 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & -c/m & 0 \\ 0 & 0 & 0 & -c/m \end{bmatrix}
 \begin{bmatrix} x \\ y \\ \dot{x} \\ \dot{y}\end{bmatrix} + 
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1/m & 0 \\ 0 & 1/m \end{bmatrix}
 \begin{bmatrix} u_x \\ u_y\end{bmatrix} $$

two 2D point masses:
$$\frac{d}{dt}\begin{bmatrix} x_1 \\ y_1 \\ \dot{x}_1 \\ \dot{y}_1 \\ x_2 \\ y_2 \\ \dot{x}_2 \\ \dot{y}_2\end{bmatrix} =  
 \begin{bmatrix} 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\ 
                0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\ 
                0 & 0 & -c/m_1 & 0 & 0 & 0 & 0 & 0\\ 
                0 & 0 & 0 & -c/m_1 & 0 & 0 & 0 & 0\\
                0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\ 
                0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\ 
                0 & 0 & 0 & 0 & 0 & 0 & -c/m_2 & 0\\ 
                0 & 0 & 0 & 0 & 0 & 0 & 0 & -c/m_2\\ \end{bmatrix}
 \begin{bmatrix} x_1 \\ y_1 \\ \dot{x}_1 \\ \dot{y}_1 \\ x_2 \\ y_2 \\ \dot{x}_2 \\ \dot{y}_2\end{bmatrix} + 
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1/m_1 & 0 \\ 0 & 1/m_1 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0\end{bmatrix}
 \begin{bmatrix} u_x^1 \\ u_y^1 \end{bmatrix} +
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 1/m_2 & 0 \\ 0 & 1/m_2\end{bmatrix}
 \begin{bmatrix} u_x^2 \\ u_y^2 \end{bmatrix}  $$


### State and input Jacobian of 2 point masses

In [35]:
m₁ = 1
m₂ = 1
c = 0
A1 = sparse([0 0 1 0; 0 0 0 1; 0 0 (-c/m₁) 0; 0 0 0 (-c/m₁)])
A2 = sparse([0 0 1 0; 0 0 0 1; 0 0 (-c/m₂) 0; 0 0 0 (-c/m₂)])
A = blockdiag(A1, A2)
B1 = sparse([0 0; 0 0; (1/m₁) 0; 0 (1/m₁); 0 0; 0 0; 0 0; 0 0])  #Control Jacobian for point mass 1
B2 = sparse([0 0; 0 0; 0 0; 0 0; 0 0; 0 0; (1/m₂) 0; 0 (1/m₂)])    #Control Jacobian for point mass 2

8×2 SparseMatrixCSC{Float64, Int64} with 2 stored entries:
  ⋅    ⋅ 
  ⋅    ⋅ 
  ⋅    ⋅ 
  ⋅    ⋅ 
  ⋅    ⋅ 
  ⋅    ⋅ 
 1.0   ⋅ 
  ⋅   1.0

### Discretize

In [93]:
n = 8 #n states
dt = 0.01 #step size
H = 15.0  #Horizon
k_steps = trunc(Int, H/dt) 

Ad = dt .* A + I    #discretize (zero order hold)
B1d = dt .*B1   #discrete (zero order hold)
B2d = dt .*B2;   #discrete (zero order hold)

In [85]:
Q1 = sparse(zeros(8,8)) #state cost for point mass 1
Q1[1,1] = 1; Q1[2,2] = 1;
Q2 = sparse(zeros(8,8))   #state cost for point mass 2
Q2[1,1] = -1; Q2[2,2] = -1;
R11 = I(2)    #Control cost for player 1
R22 = I(2)    #Contorl cost for player 2
R12 = sparse(zeros(2,2))    #Control cost for player 1 associated with player 2's controls
R21 = sparse(zeros(2,2))    #Control cost for player 2 associated with player 1's controls


2×2 SparseMatrixCSC{Float64, Int64} with 0 stored entries:
  ⋅    ⋅ 
  ⋅    ⋅ 

In [91]:
x0 = [0; 0; 1; 0; 0; 0; 0; 0]
xref = [2; 1; 1; 2; 0; 0; 0; 0]
function cost(xref, x0)
    x_k = copy(xref - x0)
    c1 = 0
    c2 = 0
    for k in length(k_steps)
        K1 = -sparse([0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0])
        K2 = -sparse([0.3 -0.1 0.2 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0])
        u1 = K1*x_k
        u2 = K2*x_k
        x_k = Ad*x_k + B1d*u1 + B2d*u2
        c1 += x_k'*Q1*x_k + u1'*R11*u1 + u2'*R12*u2
        c2 += x_k'*Q2*x_k + u2'*R22*u2 + u2'*R21*u2
    end
    return c1,c2
end

cost (generic function with 1 method)

In [92]:
c1, c2 = cost(xref, x0)

(5.1304, -4.7904)

In [8]:
# dt = 0.01
# A = [0 1; 0 -c/m]
# B = [0; 1/m]
# Ad = dt .* A + I #discrete
# Bd = dt .*B #discrete
# function lqr!(T,n,m)
#     Q = 20.0*I#[0.5 0; 0 0.3]
#     R = 1.0*I#[0.5]
#     V = copy(Q)
#     #println(T[1])
#     T[1] = T[1] #- 1
#     K = zeros(T[1], n*m)

#     for i in 1:T[1]
#         K_i = -inv(R + transpose(Bd) * V * Bd) * transpose(Bd) * V * Ad
#         #println("fff", K_i)
#         V = Q + transpose(K_i) * R * K_i + transpose(Ad + Bd * K_i) * V * (Ad + Bd * K_i)
#         K[i, :] = reshape(transpose(K_i), (n*m,1))
#         #println("f",reshape(K[i, :], (2,4))
#     end
#     #println("lqr..")
#     return K
# end

In [11]:
# tf = 15.0
# t = Array(range(0, tf, step=0.01))
# length(t)
# T_all = [1000]
# K = lqr!(T_all,n,m)
# x0 = [0.; 0.; 0. ; 0.]
# #x0 = [0.; 0.]
# #xref = [5.; 0.]
# xref = [5.; 0.; 5. ; 0.]
# #Forward rollout starting at x0
# xhist = zeros(T_all[1],n)
# xhist[1,:] = x0
# uhist = zeros(T_all[1]-1,m)
# for t = 1:(T_all[1]-1)
#     #uhist[t,:] = -K[t,:]*(xhist[t,:] - xref)
#     uhist[t,:] .= reshape(K[end, :], (m, n))*(xhist[t,:] - xref)

#     xhist[t+1,:] .= Ad*xhist[t,:] + reshape(Bd,(n,m))*uhist[t,:]
# end
#K

In [9]:
# Q = 1.0*I#[0.5 0; 0 0.3]
# R = 1.0*I#[0.5]
# Kinf = dlqr(Ad,Bd,Q,R)

In [10]:
# K = lqr!(T_all)[end, :]